#**Qual a finalidade desse notebook?**
Este nootebook tem por fim fazer alguns ajustes no **dataset** (o construído na etapa anterior), conforme interesse da pesquisa relacionada à compreensão de como certa configuração sócio-econômica favorece, ou não, a realização dos nívies de contanminação e/ou óbitos da relacionados à Covid19

#**Hands-on ...**

## **1. Carregar Libs, Carregar *dataset*, realizar ajustes & exportar**

In [1]:
# get libs

!pip install --upgrade scikit-learn
!pip install --upgrade pivottablejs

from time import sleep
import zipfile
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import plot_tree
import joblib

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

# Rich visual representation of estimators (new 0.23.2)
from sklearn import set_config
set_config(display='diagram')

     |████████████████████████████████| 6.8MB 5.1MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [23]:
#######################################
## Descompactando o arquivo do dataset
#######################################

DIR_BASE = '/content/drive/My Drive/Doutorado/Disciplinas/Aprendizado de Maquina/Primeiro Trabalho - ML/database - brasil'

with zipfile.ZipFile(DIR_BASE+'/dataset.zip','r') as zip_ref:
  zip_ref.extractall(DIR_BASE+'/')

# Globals
def isNaN(value):
  return isinstance(value, float) and math.isnan(value)

In [41]:
#########################
## Carregar o dataset
#########################

df = pd.read_csv(DIR_BASE+'/dataset.csv', sep=';')
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4643 entries, 0 to 4642
Data columns (total 74 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   IBGE                    4643 non-null   int64  
 1   NOME                    4643 non-null   object 
 2   PMATPUB_EF              4643 non-null   object 
 3   PMATPUB_EM              4643 non-null   object 
 4   DIST_EF_PUB             4643 non-null   object 
 5   DIST_EM_PUB             4643 non-null   object 
 6   IDEB_AI                 4643 non-null   object 
 7   IDEB_AF                 4643 non-null   object 
 8   DOCSUP_EF_PUB           4643 non-null   object 
 9   DOCSUP_EM_PUB           4643 non-null   object 
 10  TXMOINF                 4643 non-null   object 
 11  TXBRUTAMORT             4643 non-null   object 
 12  TXCOBPLP                4643 non-null   object 
 13  PINTERSAP               4643 non-null   object 
 14  PINTERDRSAI             4643 non-null   

In [42]:
#######################################
## Ajustes - Converter tipo de colunas
#######################################

def treat_comma(x):
  if (x.dtype == 'object' and (x.str.contains(',')).all()): 
    return x.str.replace(',','.')
  return x

df = (df.apply(treat_comma, axis = 0)).convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4643 entries, 0 to 4642
Data columns (total 74 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   IBGE                    4643 non-null   Int64  
 1   NOME                    4643 non-null   string 
 2   PMATPUB_EF              4643 non-null   string 
 3   PMATPUB_EM              4643 non-null   string 
 4   DIST_EF_PUB             4643 non-null   string 
 5   DIST_EM_PUB             4643 non-null   string 
 6   IDEB_AI                 4643 non-null   string 
 7   IDEB_AF                 4643 non-null   string 
 8   DOCSUP_EF_PUB           4643 non-null   string 
 9   DOCSUP_EM_PUB           4643 non-null   string 
 10  TXMOINF                 4643 non-null   string 
 11  TXBRUTAMORT             4643 non-null   string 
 12  TXCOBPLP                4643 non-null   string 
 13  PINTERSAP               4643 non-null   string 
 14  PINTERDRSAI             4643 non-null   

In [43]:
##################
## Ajustes Gerais
##################

unchange_columns = ['IBGE', 'estado', 'Capital', 'municipio', 'Região', 'CLASS', 'Porte']

def my_to_numeric(x):
  if (x.name in unchange_columns):
    return x
  return pd.to_numeric(x)

df.drop_duplicates(inplace=True)
df.drop(columns=['data', 'POP', 'NOME', 'POPTOT', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'], inplace=True)

df = df.apply(my_to_numeric)
df = df.rename(columns={'interior/metropolitana': 'RegMetro' })

df = df.astype({ 
  'casosAcumulado': float, 
  'obitosAcumulado': float, 
  'RegMetro': str, 
  'Capital': str, 
  'POP_TOT': float, 
  'ANNUAL': float 
})

df.loc[df['RegMetro'] == '0', 'RegMetro'] = 'Interior'
df.loc[df['RegMetro'] == '1', 'RegMetro'] = 'Região Metropolitana'

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4643 entries, 0 to 4642
Data columns (total 58 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IBGE               4643 non-null   Int64  
 1   PMATPUB_EF         4643 non-null   float64
 2   PMATPUB_EM         4643 non-null   float64
 3   DIST_EF_PUB        4643 non-null   float64
 4   DIST_EM_PUB        4643 non-null   float64
 5   IDEB_AI            4643 non-null   float64
 6   IDEB_AF            4643 non-null   float64
 7   DOCSUP_EF_PUB      4643 non-null   float64
 8   DOCSUP_EM_PUB      4643 non-null   float64
 9   TXMOINF            4643 non-null   float64
 10  TXBRUTAMORT        4643 non-null   float64
 11  TXCOBPLP           4643 non-null   float64
 12  PINTERSAP          4643 non-null   float64
 13  PINTERDRSAI        4643 non-null   float64
 14  REN_TRPCBF_D       4643 non-null   float64
 15  REN_TRPCBPC_D      4643 non-null   float64
 16  PDEFAGUA           4643 

In [45]:
###################################
## Exportar o resultado de dataset 
###################################

compression_opts = dict(method='zip', archive_name='dataset_02.csv')  
df.to_csv (DIR_BASE+'/dataset_02.zip', sep=';', index=False, header=True, compression=compression_opts)